In [1]:
!pip install python-terrier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 9.3 MB/s 
     |████████████████████████████████| 1.3 MB 51.5 MB/s 
     |████████████████████████████████| 69 kB 8.1 MB/s 
     |████████████████████████████████| 311 kB 50.8 MB/s 
     |████████████████████████████████| 46 kB 4.1 MB/s 
     |████████████████████████████████| 286 kB 81.2 MB/s 
     |████████████████████████████████| 45 kB 3.7 MB/s 
     |████████████████████████████████| 1.2 MB 62.1 MB/s 
     |████████████████████████████████| 294 kB 93.9 MB/s 
     |████████████████████████████████| 126 kB 81.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 72 kB 1.2 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.9.1-py3-none-any.whl size=110767 sha256=ba6c985c0c518bb61a7437f8a88e9ca55aff457de719

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
# Helpful for showing indexing information
pd.set_option('display.max_colwidth', 150)

import pyterrier as pt
import os

In [6]:
if not pt.started():
    pt.init()

In [19]:
df = pd.read_csv('/content/drive/MyDrive/SI650/650 Project/SI650-Data-Recommendation-vertical-search/scrapped_amazon_general_queries_data.csv', usecols=["productTitle", "productDescription", "Brand", "Ingredients", "About this item", "URL"], sep=';', header=0)
df['docno'] = 'd' + (df.reset_index().index).astype(str)
df['text'] = df[["productTitle", "productDescription", "Brand", "Ingredients", "About this item"]].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
print(df.head(5))

                                                                                                                                        productTitle  \
0                        Maybelline Hyper Easy Liquid Pen No-Skip Eyeliner Satin Finish Waterproof Formula Eye Liner Makeup Pitch Black 0.018 Fl; Oz   
1                         L'Oreal Paris Makeup Infallible Never Fail Original Mechanical Pencil Eyeliner with Built in Sharpener Black Brown 2 Count   
2  L'Oreal Paris L’Oreal Paris Makeup Infallible Grip Precision Felt Eyeliner Smudge Resistant Long Lasting Waterproof Eyeliner Grey Grey 0.03 fl oz   
3                                                                                             HotDeal Lakme Insta Deep Intensee Black Eye Liner 9 ml   
4                                                                                                      Lakme Insta Deep Intense Black Eye Liner 9 ml   

                                                                                       

In [20]:
index_dir = os.path.abspath('amazonindex')
indexer = pt.DFIndexer(index_dir, overwrite=True)
index_ref = indexer.index(df["text"], df["docno"], )
index_ref.toString()

'/content/amazonindex/data.properties'

In [21]:
index = pt.IndexFactory.of(index_ref)

In [22]:
print(index.getCollectionStatistics().toString())

Number of documents: 2801
Number of terms: 4274
Number of postings: 70394
Number of fields: 0
Number of tokens: 105257
Field names: []
Positions:   false



In [25]:
qdf = pd.read_csv("/content/drive/MyDrive/SI650/650 Project/SI650-Data-Recommendation-vertical-search/combined_queries.csv", usecols=['qid', 'query'], header=0)
print(qdf)

    qid                                        query
0     1                       facewash for oily skin
1     2                              concealer stick
2     3                      cruelty free foundation
3     4                                 gel nail kit
4     5                      frizzy hair conditioner
5     6                         waterproof eyeshadow
6     7                blush and highlighter palette
7     8                   waterproof liquid eyeliner
8     9                      matte eyeshadow palette
9    10                   lip liner and lipstick set
10   11                                nude lipstick
11   12                        skin care sets & kits
12   13                             anti aging serum
13   14          vitamin c serum for face dark spots
14   15                        bronzer stick contour
15   16              mascara black volume and length
16   17            primer face makeup pore minimizer
17   18     moisturizer with spf for acne pron

In [33]:
# Models bm25, tf_idf, pl2
rank1 = pt.BatchRetrieve(index, wmodel='BM25', num_results=50)
rank2 = pt.BatchRetrieve(index, wmodel='TF_IDF', num_results=50)
rank3 = pt.BatchRetrieve(index, wmodel='PL2', num_results=50)
final_rank = rank1 | rank2 | rank3

In [40]:
mres_final_rank = final_rank(qdf)
print(mres_final_rank)

    qid  docid  docno                    query
46    1   1082  d1082   facewash for oily skin
4     1   1093  d1093   facewash for oily skin
8     1   1112  d1112   facewash for oily skin
1     1   1115  d1115   facewash for oily skin
44    1   1200  d1200   facewash for oily skin
..   ..    ...    ...                      ...
393   9     85    d85  matte eyeshadow palette
386   9     86    d86  matte eyeshadow palette
414   9     88    d88  matte eyeshadow palette
428   9      9     d9  matte eyeshadow palette
397   9     97    d97  matte eyeshadow palette

[3297 rows x 4 columns]


In [38]:
# newdf = mres_final_rank.drop_duplicates(subset = ['docno', 'query'],
#   keep = 'last').reset_index(drop=True)
# print(newdf)

     qid  docid  docno                    query
0      1   1082  d1082   facewash for oily skin
1      1   1093  d1093   facewash for oily skin
2      1   1112  d1112   facewash for oily skin
3      1   1115  d1115   facewash for oily skin
4      1   1200  d1200   facewash for oily skin
...   ..    ...    ...                      ...
3292   9     85    d85  matte eyeshadow palette
3293   9     86    d86  matte eyeshadow palette
3294   9     88    d88  matte eyeshadow palette
3295   9      9     d9  matte eyeshadow palette
3296   9     97    d97  matte eyeshadow palette

[3297 rows x 4 columns]


In [41]:
mres_final_rank.to_csv('final_results.csv',index=False,columns=['query','docno'])

In [42]:
df.to_csv('final_reference.csv',index=False,columns=['docno','URL','text'])